# TFG: Alfonso Moure

Este fichero contiene un borrador del proyecto TFG de Alfonso Moure, donde se exploran diferentes técnicas para el trabajo con redes neuronales gráficas.

## Instalando los módulos necesarios

Como primer paso, se va a proceder a instalar los módulos necesarios para las pruebas:

* [numpy](https://numpy.org/), para poder realizar las operaciones necesarias sobre los datos.
* [TensorFlow](https://www.tensorflow.org/), como librería principal para ejecutar los distintos algoritmos de aprendizaje computacional a los que será necesario recurrir.
* [Spektral](https://graphneural.network/), una cómoda librería y colección de sets de datos que permite trabajar de manera cómoda con redes neuronales gráficas.

Una vez instalados, se procede a imporarlos. Aunque se incluye la descripción del fichero requierements.txt con el proyecto, se proceden a importar desde el notebook para aportar una mayor claridad.

In [1]:
# Install required modules
!pip install numpy
!pip install tensorflow
!pip install spektral

In [2]:
# Import downloaded modules
import numpy as np
import tensorflow as tf
import spektral

Además, se crea un fichero de configuración para Spektral en el directorio raíz del usuario, tal y como se indica en la configuración de dicho módulo. Se crea en la ruta `~/.spektral/config.json` y se guarda el siguiente contenido:

```
{
        "dataset_folder": "/Users/ghostmou/vscode-projects/uoc-tfg-gnn/spektraldata"
}
```

Esto indica a Spektral que se desea guardar la información descargada para los juegos de datos que van a ser usados en la ruta indicada.

## Carga del juego de datos para las pruebas

Como se indica en la memoria de proyecto, se hará uso del juego de datos conocido como CORA. Gracias a Spektral, es sencillo descargar este juego de datos y sus distintas estructuras ya preparadas.

In [3]:
# Download CORA dataset and its different members
dataset = spektral.datasets.citation.Citation(
    'cora', 
    random_split=False, # split randomly: 20 nodes per class for training, 30 nodes 
        # per class for validation; or "Planetoid" (Yang et al. 2016)
    normalize_x=False,  # normalize the features
    dtype=np.float32 # numpy data type for the graph data
    )
dataset.graphs[0]

# Also load a list of labels as names, justo to be able to use it
label_names = ['Case_Based', 'Genetic_Algorithms', 'Neural_Networks', 'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning', 'Theory']

/Users/ghostmou/virtualenvs/uoc-tfg-gnn/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)

Con el conjunto de datos descargado, podemos ver que el resumen nos muestra que se ha descargado un grafo con las siguientes características:

* 2708 nodos o vértices forman el grafo.
* 1433 atributos de nodo.
* 0 atributos de relación, es decir, es un nodo cuyas aristas no contienen información.
* 7 clases. En base a la definición del juego de datos, sabemos que los vértices se clasifican en dicho número de grupos. Sin embargo, como se explica en la memoria de proyecto, también es posible hacer uso de GNNs para clasificar grafos, por lo que Spektral nos permite trabajar con dos tipos de etiquetas: de nodo y de grafo.

Por otro lado, el conjunto de datos recuperado ya viene dividido en tres grupos de muestras:

* Muestras de entrenamiento, que aparecen marcadas mediante enmascaramiento con la estructura `mask_tr`.
* Muestras de validación, que hacen lo propio mediante `mask_va`.
* Muestras de prueba o test, enmascaradas con `mask_te`.

In [4]:
print(f'Training samples: {np.sum(dataset.mask_tr)}')
print(f'Validation samples: {np.sum(dataset.mask_va)}')
print(f'Test samples: {np.sum(dataset.mask_te)}')

Training samples: 140
Validation samples: 500
Test samples: 1000


## Preparación TensorBoard para seguimiento

Antes de proceder con la implementación de los ejemplos, se prepara un entorno basado en TensorBoard para poder visualizar y analizar la ejecución  y sus resultados de manera visual.

In [6]:
# Import TensorBoard callback to be able to use it in all the code samples
from keras.callbacks import TensorBoard

# Prepare placement for the logs
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir(model_in_use):
    import time
    run_id = time.strftime(f'run_{model_in_use}_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

## Implementación de pruebas

A continuación, se va a trabajar en la implementación de varios modelos diferentes de ConvGNN para explicar su funcionamiento:

* ConvGNN espectral, con un ejemplo de clasificación de nodos.
* ConvGNN espacial con paso de mensajes o MPNN, con un ejemplo de clasificación de nodos.

Cada apartado irá rematado con un estudio de precisión. Todos los casos serán ejecutados mediante GridSearch para intentar encontrar el resultado más óptimo bajo las condiciones actuales. Además, se extraerá información gráfica mediante TensorBoard.

### Implementación 1: ConvGNN espectral




Puede verse que se cuenta con 140 muestras de entrenamiento, 500 de validación y 1000 de test.

Dado que este enmascaramiento se lleva a cabo mediante datos binarios (verdadero o falso; incluido o excluido de cada subconjunto, respectivamente), es preciso transformar estos valores en pesos que puedan ser usados durante el proceso de aprendizaje: sabemos que las CNN espestrales no hacen uso de los features de nodos y vértices para su toma de decisiones.

Para ello, se crea una función que convierte estas colecciones en una media de peso de los valores de los nodos.

**ESTE BLOQUE REQUIERE REVISIÓN DESDE MIS NOTAS**

In [5]:
weighed_mask = [
    mask.astype(np.float32) / np.count_nonzero(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
]

Así, cada vértice de cada colección de muestras tendrá asignado un peso igual al del resto de su conjunto:

In [40]:
print(f'Training samples weight: {np.nanmean(np.where(weighed_mask[0] > 0, weighed_mask[0],np.nan), 0)}')
print(f'Validation samples weight: {np.nanmean(np.where(weighed_mask[1] > 0, weighed_mask[1],np.nan), 0)}')
print(f'Test samples weight: {np.nanmean(np.where(weighed_mask[2] > 0, weighed_mask[2],np.nan), 0)}')

Training samples weight: 0.0071428571827709675
Validation samples weight: 0.0020000000949949026
Test samples weight: 0.0010000001639127731


Preparados los datos, se puede proceder a definir el modelo. Para ello, se hará uso del existente GCN procedente de Spektral (`spektral.models.gcn`).

In [7]:
# Import model
from spektral.models.gcn import GCN

# Import optimizers
from tensorflow.keras.optimizers import Adam

# Import loss function from Keras
from tensorflow.keras.losses import CategoricalCrossentropy

# Set initial configuration
learning_rate = 0.01
reduction_function = 'sum'

# Create model from Spektral
model_gcn = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)

# Compile loaded model
model_gcn.compile(
    optimizer=Adam(learning_rate=learning_rate), # Set optimizer as Adam with a learning rate of 0.01
    loss=CategoricalCrossentropy(reduction=reduction_function), # Loss function to be used.
    weighted_metrics=['acc'] # Metrics to be evaluated and weighted during training (Keras doc: https://keras.io/api/models/model_training_apis/)
)

2021-11-27 12:34:39.512287: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ahora que tenemos el modelo creado y compilado, es posible proceder con su entrenamiento. Para ello, se empieza por crear los cargadores (*Loaders*) de Spektral para entrenamiento y validación, básicos para las tareas de entrenamiento. Dentro de Spektral, los *loaders* son usados para generar lotes de subgrafos para poder hacer sucesivas pasadas de entrenamiento en la red convolucional en uso. 

Gracias a que Spektral está diseñado para trabajar con Keras, la clase `Loader` incluye un método `load` que puede ser usado en para llamar al método `fit` del modelo.

Puesto que el conjunto de datos incluye un solo grafo, es posible hacer uso del cargador `SingleLoader`, diseñado para trabajar con este tipo de estructuras de grafo único. Puede ser configurado mediante los siguientes parámetros pasados durante su inicialización:

* `dataset`, que incluye, como puede deducirse, la estructura de datos.
* `epochs`, con la cantidad de **epochs** que van a ser usados durante la fase de entrenamiento.
* `shuffle`, para indicar si se desea barajar los contenidos del conjunto de datos tras cada epoch.
* `sample_weights`, que podrá ser usado para indicar el peso de cada observación y que, en caso de ser usado en la inicialización, será devuelto en cada paso de entrenamiento. Esta es la estructura que hemos generado con anterioridad para cada subconjunto de entrenamiento, pruebas y validación en base a las estructuras binarias originales. Así, se usará un vector de pesos diferente según el cargador que estemos preparando.

Puesto que el siguiente paso es llevar a cabo el entrenamiento del modelo, se prepararán los cargadores de los datos de entrenamiento y validación mediante `SingleLoader`.

In [8]:
from spektral.data.loaders import SingleLoader
loader_training = SingleLoader(dataset, sample_weights=weighed_mask[0])
loader_validation = SingleLoader(dataset, sample_weights=weighed_mask[1])

Con los cargadores listos, es posible proceder a entrenar el modelo mediante la función `fit`. A modo de prueba, se hará con un total de `50` epochs. Además, se incorpora un callback de Keras, `EarlyStopping`, que permitirá detener el entrenamiento cuando el proceso detecte que no se está obteniendo una mejora sustancial.

Como se mencionó más arriba, se incorpora también un callback `TensorBoard` para poder analizar las pruebas. Sus resultados serán accesibles mediante el siguiente comando de consola:

```shell
tensorboard --logdir=./my_logs --port=6006
```

In [9]:
# Fit the model
from tensorflow.keras.callbacks import EarlyStopping
model_gcn.fit(
    loader_training.load(),
    steps_per_epoch=loader_training.steps_per_epoch,
    validation_data=loader_validation.load(),
    validation_steps=loader_validation.steps_per_epoch,
    epochs=50,
    callbacks=[
        EarlyStopping(patience=10,  restore_best_weights=True), # Early stopping callback
        TensorBoard(get_run_logdir('gcn_spectral'))
    ]
)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 5.2150 - acc: 0.0857 - val_loss: 4.3337 - val_acc: 0.1680
Epoch 2/50
1/1 [==============================] - 0s 33ms/step - loss: 3.4113 - acc: 0.2429 - val_loss: 2.5733 - val_acc: 0.2700
Epoch 3/50
1/1 [==============================] - 0s 31ms/step - loss: 4.6410 - acc: 0.3071 - val_loss: 1.9669 - val_acc: 0.2540
Epoch 4/50
1/1 [==============================] - 0s 30ms/step - loss: 1.7633 - acc: 0.3857 - val_loss: 1.9606 - val_acc: 0.2200
Epoch 5/50
1/1 [==============================] - 0s 29ms/step - loss: 1.6963 - acc: 0.3571 - val_loss: 1.7960 - val_acc: 0.2960
Epoch 6/50
1/1 [==============================] - 0s 34ms/step - loss: 1.4287 - acc: 0.5071 - val_loss: 1.9187 - val_acc: 0.3020
Epoch 7/50
1/1 [==============================] - 0s 38ms/step - loss: 1.4357 - acc: 0.4286 - val_loss: 1.8812 - val_acc: 0.3020
Epoch 8/50
1/1 [==============================] - 0s 31ms/step - loss: 1.3676 - acc: 0.5071 - val_l

Una vez hecho este entrenamiento, se puede proceder a evaluar su eficacia.

In [10]:
loader_test = SingleLoader(dataset, sample_weights=weighed_mask[2])
results = model_gcn.evaluate(
    loader_test.load(), 
    steps=loader_test.steps_per_epoch
)
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')

1/1 [==============================] - 0s 10ms/step - loss: 1.3889 - acc: 0.6890
Loss: 1.388916015625
Accuracy: 0.6890002489089966


### Implementación 2: ConvGNN espacial mediante paso de mensajes: MPNN

En el ejemplo anterior se ha basado el aprendizaje y la clasificación en la estructura espectral de los datos cargados, es decir, en la esutructura de la matriz de adyacencia. Sin embargo, tal y como se explica en la memoria de proyecto, esta aproximación puede ser pobre para escenarios donde las relaciones (aristas) entre entidades (vértices) del grafo tienen un significado o importancia diferente, o cuando el contexto de un nodo, construido mediante los atributos de sus vecinos, es importante para la clasificación.

Para poder preparar este ejemplo, se hará uso de la clase `MessagePassing` de Spektral, que ofrece una API ya preparada para configurar la función de activación y personalizar el comportamiento para distintos juegos de datos. Así, será preciso personalizar:

* Función para la construcción del mensaje pasado entre dos vértices vía la arista que los une. Es conocida como `message` dentro de la API de Spektral.
* Selección de la función de agregación de los mensajes pasados desde cada arista a cada nodo: suma, media, etc. Aparece definida como `aggregate`.

Puesto que las GNN basadas en paso de mensajes requieren sucesivas iteraciones que permitan propagar los mensajes a niveles cada vez más lejanos, la función `propagate` lo ejecuta y computa los atributos de cada nodo tras pasar los mensajes de todas las aristas del grafo y computar la correspondiente función de agregación.

Con todo, se empieza creando la clase MPNNLayer como herencia `MessagePassing` para preparar la personalización de los citados métodos y definir la capa de paso de mensajes del modelo a usar.

In [11]:
from spektral.layers import MessagePassing

# TODO Refactor this!

class MPNNLayer(MessagePassing):
    def __init__(self, n_out, activation, **kwargs):
        # Initialize message passing layer with the activation function chosen when creating the model
        super().__init__(activation=activation, **kwargs)
        self.n_out = n_out

    def build(self, input_shape):
        n_in = input_shape[0][-1]
        self.weights = self.add_weight(shape=(n_in, self.n_out))

    def call(self, inputs, **kwargs):
        x, a = inputs

        # Update node features based on inputs by multiplying node attributes by 
        # the weights stored during build.
        # https://www.tensorflow.org/api_docs/python/tf/linalg/matmul
        x = tf.matmul(x, self.weights)

        # Return propagation result
        return self.propagate(x=x, a=a)

    def message(self, x, **kwargs):
        # We can access information from each side of the link between the nodes i and j: j <-- j
        # edge indices: index_i, index_j attributes
        # access edges: get_i, get_j methods

        # Try to return neighbors' features
        return self.get_j(x)
        # Try to return node features
        # return self.get_i(x)
    
    def aggregate(self, messages, **kwargs): 
        # We need to return the result of applying the aggregate function over the messages
        # TODO Possible improvement: pass the aggregate function as an hyperparameter to the layer to test different solutions

        # Try to use the mean as aggregate function. We use a scatter mean method for this experiment:
        return spektral.layers.ops.scatter_mean(messages, self.index_i, self.n_nodes)

    def update(self, embeddings, **kwargs):
        return self.activation(embeddings)


**Nota para MOU**: me he quedado intentando construir una layer de MPNN en base a lo que leo aquí:

https://graphneural.network/creating-layer/

Estoy cansado y no me da para más, pero creo que tienes que crear el modelo con Keras y 

In [12]:
# MPNNLayer(n_out=dataset.n_labels, n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features, activation=tf.keras.activations.relu)


### Implementación 3: clasificación de nodos mediante CNN sin uso de estructura gráfica

El primer paso será construir una estructura de datos que nos permita trabajar con un modelo que no esté diseñado para operar sobre grafos. Para ello, se extraerán las características de cada observación de los datos de origen y no se usarán sus enlaces. Con todo, el objetivo es medir el nivel de precisión a la hora de clasificar sin utilizar la estructura generado mediante las relaciones entre nodos.

In [92]:
# Extract train, validation and test features and labels
X_train = dataset[0].x[np.array(dataset.mask_tr)]
y_train = dataset[0].y[np.array(dataset.mask_tr)]
X_validation = dataset[0].x[np.array(dataset.mask_va)]
y_validation = dataset[0].y[np.array(dataset.mask_va)]
X_test = dataset[0].x[np.array(dataset.mask_te)]
y_test = dataset[0].y[np.array(dataset.mask_te)]

print(f'Training: X {X_train.shape}, y {y_train.shape}, from a source of {np.sum(dataset.mask_tr)} samples')
print(f'Validation: X {X_validation.shape}, y {y_validation.shape}, from a source of {np.sum(dataset.mask_va)} samples')
print(f'Test: X {X_test.shape}, y {y_test.shape}, from a source of {np.sum(dataset.mask_te)} samples')


Training: X (140, 1433), y (140, 7), from a source of 140 samples
Validation: X (500, 1433), y (500, 7), from a source of 500 samples
Test: X (1000, 1433), y (1000, 7), from a source of 1000 samples


In [128]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=X_train[0].shape),
    keras.layers.Dense(300, activation=keras.activations.relu),
    keras.layers.Dense(7, activation=keras.activations.relu)
])

# Compile model (based on the same parameters used with the espectral ConvGNN)
model.compile(
    optimizer=Adam(learning_rate=0.01), # Set optimizer as Adam with a learning rate of 0.01
    loss=CategoricalCrossentropy(reduction=reduction_function), # Loss function to be used.
    weighted_metrics=['acc'] # Metrics to be evaluated and weighted during training (Keras doc: https://keras.io/api/models/model_training_apis/)
)

model.fit(
    X_train, y_train, epochs=30, 
    validation_data=(X_validation, y_validation),
    callbacks=[
        EarlyStopping(patience=10,  restore_best_weights=True), # Early stopping callback
        TensorBoard(get_run_logdir('non_gnn'))
    ]
)

Epoch 1/30
5/5 [==============================] - 1s 54ms/step - loss: 275.7899 - acc: 0.2214 - val_loss: 183.8702 - val_acc: 0.3520
Epoch 2/30
5/5 [==============================] - 0s 19ms/step - loss: 180.6099 - acc: 0.5214 - val_loss: 162.0301 - val_acc: 0.3980
Epoch 3/30
5/5 [==============================] - 0s 20ms/step - loss: 154.3290 - acc: 0.6571 - val_loss: 140.4636 - val_acc: 0.4480
Epoch 4/30
5/5 [==============================] - 0s 20ms/step - loss: 149.6595 - acc: 0.7000 - val_loss: 139.1511 - val_acc: 0.4700
Epoch 5/30
5/5 [==============================] - 0s 23ms/step - loss: 149.6093 - acc: 0.7071 - val_loss: nan - val_acc: 0.4800
Epoch 6/30
5/5 [==============================] - 0s 15ms/step - loss: 141.2671 - acc: 0.7071 - val_loss: nan - val_acc: 0.4860
Epoch 7/30
5/5 [==============================] - 0s 20ms/step - loss: 142.7150 - acc: 0.7071 - val_loss: nan - val_acc: 0.4900
Epoch 8/30
5/5 [==============================] - 0s 30ms/step - loss: 143.5810 - ac

In [129]:
results = model.evaluate(X_test, y_test)
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')

32/32 [==============================] - 0s 2ms/step - loss: 139.1920 - acc: 0.4940
Loss: 139.19204711914062
Accuracy: 0.49399998784065247
